# Second Homework
## Dimitar Ilev
## 1. Network optimization - Multi commodity problem
# Operation Barbarossa and the Backup
(Imaginative scenario)

On 22 June 1941, Germany began a major attack on the Soviet Union, the communist state that consisted of Russia and a number of neighbouring countries. During the first nine months of the advance, one million German soldiers were killed and many we wounded. 

The German government has been loosing a lot of money due to the lenght of the war. What was expected to be a quick victory over the soviet state turned into a long lasting battle. Therefore, the government needs to find the cheapest way of transporting its backup units for the occupation of Leningrad, Moscow and Stalingrad. For the transition of the units there are the two cities Warschau and Budapest where the soldiers can regroup before continuing to their final destination.

<img src="operation-barbarossa.png" alt="Drawing" style="width: 650px;"/>

For the transport of the different army units the generals have estimated the costs per soldier per route between the cities of origin, the transitional points and the final destinations. Through communication with the front lines they have gotten requests for few hundred thousand soldiers, which is 75.000 to Leningrad, 60.000 to Moscow and 90.000 to Stalingrad. 

They know that it is not strategic to generate this amount of soldiers from only one army base in big german cities. But it is smarter to collectively put the necessary number together from the two or three major military bases in the east of the empire: Berlin, Leipzig and Munic. To have a better understanding of the situatuion the generals have created an overview including the transportation costs per soldier in Reichsmark(RMARK), possible capacities(at each location) and the demands.

<img src="OverviewBarbarossa1.png" alt="Drawing" style="width: 699px;"/>

# Mathematical definition
### Decision Variable
- Flows = x

### Parameters
- Costs = c
- Arcs = L
- Commodities (Soldiers) = k
- Nodes (Cities) = n
- Demand = Dem
- Origin = O
- Destination = D
### Objective function: 
- Sums of costs multiplied with number of flows resulting in the total costs, that the model is minimizing.
$$ minC = \sum_{k\epsilon K}^{} \sum_{m\epsilon L}^{} c_{m}^{k} \times x_{m}^{k} $$

### Set of Constraints:
$$ x_{m}^{k} \epsilon \mathbb{Z}_{0}^{+} $$
- Origin node: Positive demand, since noce is the origin of the commodities.
$$ \sum_{m\epsilon L_{i}^{out}}x_{m}^{k} - \sum_{m\epsilon L_{i}^{in}} x_{m}^{k} = Dem^{k} \rightarrow i \epsilon O(k) $$

- Destination node: Expects input and hence the demand is negative.
$$ \sum_{m\epsilon L_{i}^{out}}x_{m}^{k} - \sum_{m\epsilon L_{i}^{in}} x_{m}^{k} = -Dem^{k} \rightarrow i \epsilon D(k) $$

- Transition node: Input has to be equal to output since it is only a transitional point.
$$ \sum_{m\epsilon L_{i}^{out}}x_{m}^{k} - \sum_{m\epsilon L_{i}^{in}} x_{m}^{k} = 0 \rightarrow i \epsilon N, k \epsilon K $$

- Capacity constraint: The flow at each node has to be less equal to the set capacity of that node
$$ \sum_{k \epsilon K} x_{m}^{k} \leq  Cap_{m} \rightarrow m \epsilon L $$

### Data input

In [92]:
%%writefile mincost.dat
# Nodes set
set N := Berlin Leipzig Munic Warschau Budapest Leningrad Moscow Stalingrad;
# Arcs set
set A := (Berlin, Warschau) (Berlin,Budapest) (Leipzig, Warschau) (Leipzig, Budapest) (Munic, Warschau) (Munic, Budapest) (Warschau, Leningrad) (Warschau, Moscow) (Warschau, Stalingrad) (Budapest, Leningrad) (Budapest, Moscow) (Budapest, Stalingrad);

param origin := Berlin Leipzig Munic;
#param destination := Leningrad Moscow Stalingrad;
#param transition := Warschau Budapest;

# List of estimated costs for the transportation of one unit between each of the nodes
# Transport costs in RMARK per soldier
param: costs:= 
Berlin Warschau 3.4
Leipzig Warschau 2.1
Munic Warschau 1.3

Berlin Budapest 4.2
Leipzig Budapest 2.5
Munic Budapest 1.2

Warschau Leningrad 4.1
Budapest Leningrad 4.5

Warschau Moscow 4.8
Budapest Moscow 3.5

Warschau Stalingrad 5.1
Budapest Stalingrad 3.8;

# The capacity at each arc in thousand units 
param: capacity:= 
Berlin Warschau 65000
Leipzig Warschau 50000
Munic Warschau 45000

Berlin Budapest 45000
Leipzig Budapest 65000
Munic Budapest 70000

Warschau Leningrad 50000
Budapest Leningrad 45000

Warschau Moscow 55000
Budapest Moscow 50000

Warschau Stalingrad 60000
Budapest Stalingrad 75000;

# The demand of soldiers at each target
param: demand:=
Leningrad 75000
Moscow 60000
Stalingrad 90000;

Overwriting mincost.dat


### The Model

In [93]:
%%writefile mincost.py
# Inports
from pyomo.core import Constraint
from pyomo.environ import *

# Create an instance of the model
model = AbstractModel()

# Set Nodes list
model.N = Set()
# Set Arcs
model.A = Set(within = model.N * model.N)

# Set origin
model.origin = Param(within = model.N)
# Set destination
#model.destination = Param(model.N)
# Set transition
#model.transition = Param(model.A)
# Set costs
model.costs = Param(model.A)
# Set capacity
model.capacity = Param(model.A)
# Set demand
model.demand = Param(model.N)

# Set the decission variables => number of flows
model.flow = Var(model.A,within=NonNegativeReals)

# Define objective function
def minCost_rule(model):
    '''
    Defines the objective functions, which ensure that the objective of the model is properly calculated. The constraint multiplies the cost of the chosed arc with the flow of the commodities. The function is defined over the set of costs and flows using the Objective class.
    '''
    return sum((model.flow[i,j] * model.costs[i,j]) for (i,j) in model.A)
model.minCost = Objective(rule=minCost_rule, sense=minimize)

#Define minimum demand constraint
def demand_rule(model,i):
    '''
    Defines the demand constraints, which ensure that the final Nodes receives the necessary amount of units. This constraint is defined over the set of nodes using the Constraint class.
    '''
    if i == "Leningrad" or i == "Moscow" or i == "Stalingrad":
        return sum(model.flow[s,d] for (s,d) in model.A if d==i)>=model.demand[i]
    else: 
        return Constraint.Skip
model.demandConstraint = Constraint(model.N, rule=demand_rule)

# Create the capacity constraints
def capacity_rule(model,i,j):
    '''
    Defines the capacity constraints, which ensure that the flow on each arc does not exceed the capacity of that arc. This constraint is defined over the set of arcs (model.A) using the Constraint class.
    '''
    return model.flow[i,j] <= model.capacity[i,j]

model.capacityConstraint = Constraint(model.A, rule=capacity_rule)


# Create the flow conservation constraints
def conservation_rule(model, node):
    '''
    Defines the flow conservation constraints, which ensure that flow is conserved at each node. This constraint is defined over the set of nodes (model.N) using the Constraint class.
    '''
    if node == "Berlin" or node == "Laipzig" or node == "Munic":
        return Constraint.Skip
    if node == "Leningrad" or node == "Moscow" or node == "Stalingrad":
        #return sum(model.flow[arc] for arc in model.A if arc[1] == node) == 1
        return Constraint.Skip
    # Incoming node
    incoming = sum(model.flow[i,j] for (i,j) in model.A if node==i) # in
    # Outgoing node
    outgoing = sum(model.flow[i,j] for (i,j) in model.A if node==j) # out
    return incoming == outgoing

model.conservationConstraint = Constraint(model.N, rule=conservation_rule)

Overwriting mincost.py


In [94]:
!pyomo solve --solver=glpk mincost.py mincost.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.01] Creating model
[    0.02] Applying solver
[    0.10] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 1500000.0
    Solver results file: results.yml
[    0.10] Applying Pyomo postprocessing actions
[    0.10] Pyomo Finished


## SOLVER RESULTS

  Lower bound: 1500000.0  ---  Upper bound: 1500000.0  ---  Number of objectives: 1

  Number of constraints: 19 ---  Number of variables: 13 ---  Number of nonzeros: 33

Solution Objective minCost: 1.500.000 RMARK

  Variables:
  
    flow[Berlin,Budapest]:
      Value: 45000
    
    flow[Berlin,Warschau]:
      Value: 65000
    
    flow[Budapest,Leningrad]:
      Value: 25000
    
    flow[Budapest,Moscow]:
      Value: 15000
    
    flow[Budapest,Stalingrad]:
      Value: 75000
    
    flow[Munic,Budapest]:
      Value: 70000
    
    flow[Munic,Warschau]:
      Value: 45000
    
    flow[Warschau,Leningrad]:
      Value: 50000
    
    flow[Warschau,Moscow]:
      Value: 45000
    
    flow[Warschau,Stalingrad]:
      Value: 15000


## Conclusion
As we can see in the solver output the total minimum costs for the transportation of the soldiers is 1.500.000,00 RMARK. For the number of soldiers we can see the most optimal way to save money between the locations, thus the amount that has to be send. 

The lower and upper bound account to the same value of 1.500.000 which is great, this means that we have generated the necessary number of flows to move from city to city and thus generated the minnimum cost value. We have used a total number of 19 constraints and 13 variables. 

- There are 65.000 soldiers going from Berlin to Warschau and 45.000 to Budapest.
- Munich sends 45.000 to Warschau and 70.000 to Budapest.
- Leipzig has been compromized in this solution.
- Warschau sends 50.000 to Leningrad, 45.000 to Moscow and 15.000 to Stalingrad.
- Budapest sends 25.000 to Leningrad, 15.000 to Moscow and 75.000 to Stalingrad.

Through these combinations we get the required demand at each of the final locations.

<img src="FinalBarbarossa.png" alt="Drawing" style="width: 700px;"/>

### Interpretation
As we can see in the final outcome graphic, we reach the total demand by transporting the units from the two origins Berlin and Munich to the two transition points Warschau and Budapest. There we split up the units between the three destinations.
The model has estimated the best way of moving the provided number of units at each arc at the minimum cost possible. The set constrains have been respected and we do not exceed the maximum capacity at any of the arcs.

The reason for not sending any troops out from Leipzig is due to the transportation costs. It is cheaper to transport troops from Munic but then to reach the necessary capacity at the transition nodes the rest of the troops have to come from Berlin. Here we have a conflict between the objective and the capacity constraint, since we are trying to minimize the costs but we are also trying to reach the requested capacity at each node in order to satisfy the final demand. Hence, based on the given specifications the model has decided that it is cheapest to skip Leipzig to reach the objective locations, in order to satisfy the demand constraint. Even if the transport costs from Leipzig are way less than the costs from Berlin.

This solution satisfies the generals, as they can save money for further special military operations on the west front.

If it is enough to win the war on the east front we will have to see...

## Non-linear optimization problem